In [15]:
import os
import torch
from torchaudio import transforms
import numpy as np
import pandas as pd
import random
from torch.utils.data import DataLoader, Dataset
import torchaudio
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
current_device = torch.cuda.get_device_name(0) if torch.cuda.device_count() > 0 else 'cpu'
print(f'using device: {current_device}')

using device: GeForce GTX 1060 3GB


In [4]:
input_path = './input/cleaned/'
output_path = './input/cleaned/output/'

In [5]:
input_li = pd.Series([input_path + 'x/' + i for i in os.listdir(input_path + 'x/')], dtype=str)
# targets have the exact same name, but are in the y/ folder, so to ensure that everything is in the 
# proper order, list through the input directory, but link to the y/ directory
target_li = pd.Series([input_path + 'y/' + i for i in os.listdir(input_path + 'x/')], dtype=str)
# table = pd.DataFrame(data=[X, y], columns=['input', 'target'])
# table.head()
df = pd.DataFrame(data={'input': input_li, 'target': target_li})
print(f'missing values: {df.isna().any().sum()}')
df.head()

missing values: 0


,input,target
0,./input/cleaned/x/12 Comics You Need to See - ...,./input/cleaned/y/12 Comics You Need to See - ...
1,./input/cleaned/x/12 Comics You Need to See - ...,./input/cleaned/y/12 Comics You Need to See - ...
2,./input/cleaned/x/12 Comics You Need to See - ...,./input/cleaned/y/12 Comics You Need to See - ...
3,./input/cleaned/x/12 Comics You Need to See - ...,./input/cleaned/y/12 Comics You Need to See - ...
4,./input/cleaned/x/12 Comics You Need to See - ...,./input/cleaned/y/12 Comics You Need to See - ...


In [6]:
class AudioDataset(Dataset):
    
    def __init__(self, input_path, transform=None):
        self.input_path = input_path
        # use a root path that branches into x/ and y/ directories
        input_li = pd.Series([input_path + 'x/' + i for i in os.listdir(input_path + 'x/')], dtype=str)
        # targets have the exact same name, but are in the y/ folder, so to ensure that everything is in the 
        # proper order, list through the input directory, but link to the y/ directory
        target_li = pd.Series([input_path + 'y/' + i for i in os.listdir(input_path + 'x/')], dtype=str)
        self.df = pd.DataFrame(data={'input': input_li, 'target': target_li})
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        x = self.df['input'].iloc[idx]
        x = torchaudio.load(x)
        y = self.df['target'].iloc[idx]
        y = torchaudio.load(y)
        

In [8]:
example_path = input_path + 'x/' + \
'12 Comics You Need to See - Comedy Central Stand-Up Presents.mp3.wavclip120000version1.wav'
example_path

'./input/cleaned/x/12 Comics You Need to See - Comedy Central Stand-Up Presents.mp3.wavclip120000version1.wav'

In [11]:
audio, sample_rate = torchaudio.load(example_path)
audio.shape

torch.Size([2, 1323000])

In [ ]:
# build loss based on https://openaccess.thecvf.com/content_CVPR_2019/papers/Yuan_Signal-To-Noise_Ratio_A_Robust_Distance_Metric_for_Deep_Metric_Learning_CVPR_2019_paper.pdf


In [13]:
class Model(nn.Module):
    
    def __init__(self):
        